 Import Necessary libraries

In [1]:
import pymongo
import psycopg2
import numpy as np
import pandas as pd
import pandas.io.sql as sqlio
import urllib.request
import requests
import urllib.parse
import re

Connect to mongodb installed on ec2 instance

In [6]:
client = pymongo.MongoClient('mongodb://3.250.4.75:27017/')
# database creating in Mongodb
mydb = client['dap_project']
# creating collections in database 'dap_project'
collection_tv_reviews = mydb['dap_tv_reviews']
collection_tv = mydb['dap_tv_shows']
collection_movies_review = mydb['dap_movies_review']
collection_movies = mydb['dap_movies']

In [39]:
# Fetching details of the TV Reviews from TMDB API and storing in Mongodb
# create collection for TV Reviews data
collection_tv_reviews = mydb['dap_tv_reviews']

tv_review = 'https://api.themoviedb.org/3/tv/{}/reviews?api_key=de73d0fe4ff6171d74ac88d817fc443a&language=en-US'
tv_review_final_ids = []
for i in range(1,30000):
    try:
        tv_review_api = tv_review.format(i)
        res = requests.get(tv_review_api)
        res = res.json()
        if res['total_pages'] == 1:
            collection_tv_reviews.insert_one(res)
            tv_review_final_ids.append(i)
    except:
        continue
        
print("TV Shows Review data fetched and stored in Mongodb, good job!!")

            

TV Shows Review data fetched and stored in Mongodb, good job!!


In [41]:
# Fetching TV shows data using TMBD API and storing it in dap_project database
# create separate collection for TV Shows
collection_tv = mydb['dap_tv_shows']

tv_api = 'https://api.themoviedb.org/3/tv/{}?api_key=de73d0fe4ff6171d74ac88d817fc443a&language=en-US'
for i in range(1,30000):
    try:
        tv_api_id = tv_api.format(i)
        response_tv = requests.get(tv_api_id)
        if response_tv.status_code == 200:
            response_tv = response_tv.json()
            collection_tv.insert_one(response_tv)
    except:
        continue
        
print("Tv Shows data has been imported and stored in Mongodb, good job!!")

Tv Shows data has been imported and stored in Mongodb, good job!!


In [45]:
# Fetching Movie Reviews data using TMBD API and storing it in dap_project database
# create separate collection for Movie Reviews
collection_movies_review = mydb['dap_movies_review']

movie_review='https://api.themoviedb.org/3/movie/{}/reviews?api_key=de73d0fe4ff6171d74ac88d817fc443a&language=en-US&page=1'
movie_review_ids = []
for i in range(1, 10000):
    try:
        movie_review_api=movie_review.format(i)
        res = requests.get(movie_review_api)
        res = res.json()
        if res['total_pages'] == 1:
            collection_movies_review.insert_one(res)
            movie_review_ids.append(i)
    except:
        continue
print("Movies review has been imported and stored in Mongodb, good job!!")

Movies review has been imported and stored in Mongodb, good job!!


In [48]:
# Fetching Movies data from TMDB API and storing in Mongodb
# collection creation in database dap_project
collection_movies = mydb['dap_movies']

url = "https://api.themoviedb.org/3/movie/{}?api_key=de73d0fe4ff6171d74ac88d817fc443a&language=en-US"
for i in movie_review_ids:
    try:
        final_url = url.format(i)
        response = requests.get(final_url)
        if response.status_code == 200:
            response = response.json()
            collection_movies.insert_one(response)
    except:
        continue
        
print('Data has been imported and stored in Mongodb, good job!!') # 1104


Data has been imported and stored in Mongodb, good job!!


Connect to Postgres so that we can import data from Mongodb

In [15]:
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "3.250.4.75",
    port = "5432",
    database = "postgres")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute('CREATE DATABASE dap_project;')
    print("Database created sucessfully in postgres")
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)

Database created sucessfully in postgres


In [14]:
# use dap_project database and create tables in it
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "3.250.4.75",
    port = "5432",
    database = "dap_project")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
    
# create table in database dap_project for movie reviews
table_cloumns = """
CREATE TABLE movies_reviews(
    id integer PRIMARY KEY,
    author VARCHAR(255) NOT NULL,
    content VARCHAR(100000),
    review_id VARCHAR(1000),
    url VARCHAR(1000)
);
"""
try:
    dbCursor.execute(table_cloumns)  
    print("Table created sucessfully")
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
# creating string for movie reviews to insert into table
insert_string = "INSERT INTO movies_reviews VALUES ({mid}, '{author}', '{content}','{review_id}', '{url}')"
for j in collection_movies_review.find():
    try:
        movie_id = j['id']
        movie_author = j['results'][0]['author']
        content_data = j['results'][0]['content']
        movie_content = re.sub(r"\W+|_", " ", content_data)
        movie_review_id = j['results'][0]['id']
        movie_url = j['results'][0]['url']
        insert_string1 = insert_string.format(mid=movie_id, author=movie_author, content = movie_content,
                                                  review_id= movie_review_id, url=movie_url)
        dbCursor.execute(insert_string1)
    except:
        print(movie_id)
    finally:
        if(dbConnection): dbConnection.close()
print("Data inserted into tables successfully")

Data inserted into tables successfully


In [13]:
#create table for movies data

table_cloumn_movies = """
CREATE TABLE movies(
    id integer PRIMARY KEY,
    genre VARCHAR(255) NOT NULL,
    production_company VARCHAR(100000),
    production_country VARCHAR(10000),
    release_date timestamp,
    runtime integer,
    languages VARCHAR(10000),
    title VARCHAR(10000),
    vote_average numeric(4,2),
    vote_count integer
    );
"""

try:
    dbCursor.execute(table_cloumn_movies)
    print('Table created successfully')
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)

#inserting movies data into table from Mongodb
insert_string_movie = "INSERT INTO movies VALUES({mid}, '{genre}', '{production_company}', '{production_country}', '{release_date}', {runtime},'{langauges}','{title}',{vote_average},{vote_count})"

for i in collection_movies.find():
    try:
        movie_id = i['id']
        movie_genre = i['genres'][0]['name']
        movie_production_company=i['production_companies'][0]['name']
        movie_production_country = i['production_countries'][0]['name']
        movie_relase_date = pd.to_datetime(i['release_date'])
        movie_runtime = i['runtime']
        movie_language = i['spoken_languages'][0]['name']
        content_title = i['title']
        movie_title = re.sub(r"\W+|_", " ", content_title)
        movie_vote_average = i['vote_average']
        movie_vote = i['vote_count']
        insert_string_movie1 = insert_string_movie.format(mid=movie_id, genre=movie_genre, production_company=movie_production_company, production_country= movie_production_country, release_date=movie_relase_date, runtime=movie_runtime, langauges=movie_language, title=movie_title, vote_average=movie_vote_average, vote_count=movie_vote)
        dbCursor.execute(insert_string_movie1)
    except:
         print("Please look for the error")
    finally:
        if(dbConnection): dbConnection.close()
        
print("Data inserted into table movies")
    

Data inserted into table movies


In [10]:
#create table for tv reviews data
table_cloumns_tv_review = """
CREATE TABLE tv_reviews(
    id integer PRIMARY KEY,
    author VARCHAR(255) NOT NULL,
    content VARCHAR(100000),
    review_id VARCHAR(1000),
    url VARCHAR(1000)
);
"""
try:
    dbCursor.execute(table_cloumns_tv_review)  
    print("Table created sucessfully")
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
# inserting tv_reviews data in the Mongodb
insert_string = "INSERT INTO tv_reviews VALUES ({tid}, '{author}', '{content}','{review_id}', '{url}')"
for j in collection_tv_reviews.find():
    try:
        tv_id = j['id']
        tv_author = j['results'][0]['author']
        content_data = j['results'][0]['content']
        tv_content = re.sub(r"\W+|_", " ", content_data)
        tv_review_id = j['results'][0]['id']
        tv_url = j['results'][0]['url']
        insert_string1 = insert_string.format(tid=tv_id, author=tv_author, content = tv_content,
                                                  review_id= tv_review_id, url=tv_url)
        dbCursor.execute(insert_string1)
    except:
        print("Please look for the error")
    finally:
        if(dbConnection): dbConnection.close()
        
print("Data inserted into table tv reviews")

Error while connecting to PostgreSQL relation "tv_reviews" already exists

Data inserted into table tv reviews


In [16]:
#create table for tv shows data
table_cloumns_tv_shows = """
CREATE TABLE tv_shows(
    id integer PRIMARY KEY,
    genre VARCHAR(255) NOT NULL,
    production_company VARCHAR(100000),
    vote_average numeric(1000),
    runtime integer,
    languages VARCHAR(1000)
);
"""
try:
    dbCursor.execute(table_cloumns_tv_shows)  
    print("Table created sucessfully")
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
# inserting tv reviews data into table
insert_string_tv= "INSERT INTO tv_shows VALUES({tvid}, '{genre}', '{production_company}', {vote_average}, {runtime},'{languages}')"
for k in collection_tv.find():
    try:
        tv_review_ids = k['id']
        tv_genre = k['genres'][0]['name']
        tv_production_company = k['production_companies'][0]['name']
        tv_vote_average = k['vote_average']
        tv_runtime = k['episode_run_time'][0]
        tv_language = k['languages'][0]
        insert_string_tv1 = insert_string_tv.format(tvid=tv_review_ids, genre=tv_genre, production_company=tv_production_company, vote_average=tv_vote_average, runtime= tv_runtime, languages=tv_language)
        dbCursor.execute(insert_string_tv1)
    except:
        continue
    finally:
        if(dbConnection): dbConnection.close()
print("Data inserted sucessfully into table")

Table created sucessfully
Data inserted sucessfully into table
